# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa para analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [120]:
import pandas as pd
import numpy as np
import json
treinamento = pd.read_excel('tweetsmarlboro.xlsx')
treinamento = treinamento.loc[:,["Treinamento","Relevante"]]
treinamento.head()
relevantes = treinamento[(treinamento['Relevante']==1)]
irrelevantes = treinamento[(treinamento['Relevante']==0)]

In [121]:
treinamento.head()

,Treinamento,Relevante
0,rt @1990sf1: ayrton senna (marlboro mclaren-ho...,1.0
1,nws alert&gt; flood advisory: affected locatio...,0.0
2,rt @1990sf1: ayrton senna (marlboro mclaren-ho...,1.0
3,ciggie jacket look from yesterday\n#marlboro #...,1.0
4,at least four marlboro men have died of smokin...,1.0


In [122]:
relevantestring = " ".join(list(relevantes.Treinamento))
relevanteslimpo = relevantestring.replace('.','')
relevanteslimpo = relevanteslimpo.replace(',','')
relevanteslimpo = relevanteslimpo.replace(';','')
relevanteslimpo = relevanteslimpo.replace(':','')
relevanteslimpo = relevanteslimpo.replace('"','')
relevanteslimpo = relevanteslimpo.replace('!','')
relevanteslimpo = relevanteslimpo.replace('(','')
relevanteslimpo = relevanteslimpo.replace(')','')
relevanteslimpo = relevanteslimpo.replace('?','')
relevanteslimpo = relevanteslimpo.replace('*','')

Relevantes = pd.Series((relevanteslimpo).split()).value_counts()
Relevantes.head()

marlboro    117
rt           49
the          36
a            32
i            29
dtype: int64

In [123]:
irrelevantestring = " ".join(list(irrelevantes.Treinamento))
irrelevanteslimpo = irrelevantestring.replace('.','')
irrelevanteslimpo = irrelevanteslimpo.replace(',','')
irrelevanteslimpo = irrelevanteslimpo.replace(';','')
irrelevanteslimpo = irrelevanteslimpo.replace(':','')
irrelevanteslimpo = irrelevanteslimpo.replace('"','')
irrelevanteslimpo = irrelevanteslimpo.replace('!','')
irrelevanteslimpo = irrelevanteslimpo.replace('(','')
irrelevanteslimpo = irrelevanteslimpo.replace(')','')
irrelevanteslimpo = irrelevanteslimpo.replace('?','')
irrelevanteslimpo = irrelevanteslimpo.replace('*','')
Irrelevantes = pd.Series((irrelevanteslimpo).split()).value_counts()
Irrelevantes.head()

marlboro    146
md          115
rt          110
the          98
a            52
dtype: int64

In [124]:
Irreaspera = Irrelevantes.sum()
Releaspera = Relevantes.sum()
Irreaspera

3382

In [125]:
for p in Irrelevantes.index:
    Irrelevantes[p]+=1

In [126]:
irretotal = Irrelevantes.sum()
irretotal

4387

In [127]:
dfi = Irrelevantes.to_frame()
dfi.columns = ["Quantidade"]

In [128]:
dfi["prob"] = dfi["Quantidade"]/irretotal

In [129]:
dfi.head()

,Quantidade,prob
marlboro,147,0.033508
md,116,0.026442
rt,111,0.025302
the,99,0.022567
a,53,0.012081


In [130]:
for p in Relevantes.index:
    Relevantes[p]+=1

dfre = Relevantes.to_frame()
dfre.columns = ["Quantidade"]
dfre["prob"] = dfre["Quantidade"]/Relevantes.sum()
dfre.head()

,Quantidade,prob
marlboro,118,0.044478
rt,50,0.018847
the,37,0.013946
a,33,0.012439
i,30,0.011308


In [131]:
listatotal = list(dfre.index)+list(dfi.index)
totaldif = len(set(listatotal))

In [132]:
trele = len(list(dfre.index))
tirre = len(list(dfi.index))
c_rele = trele/totaldif
c_irre = tirre/totaldif

In [133]:
def classificador(a):
    probreli = []
    probeirri = []
    palavratweetsLimpo = a.replace('.','')
    palavratweetsLimpo = palavratweetsLimpo.replace(',','')
    palavratweetsLimpo = palavratweetsLimpo.replace(';','')
    palavratweetsLimpo = palavratweetsLimpo.replace(':','')
    palavratweetsLimpo = palavratweetsLimpo.replace('"','')
    palavratweetsLimpo = palavratweetsLimpo.replace('!','')
    palavratweetsLimpo = palavratweetsLimpo.replace('(','')
    palavratweetsLimpo = palavratweetsLimpo.replace(')','')
    palavratweetsLimpo = palavratweetsLimpo.replace('?','')
    palavratweetsLimpo = palavratweetsLimpo.replace('*','')
    palavratweets = palavratweetsLimpo.split(' ')
    for e in palavratweets:
        if e not in dfre.index:
            probreli.append(1/(Releaspera + totaldif))
            
        if e not in dfi.index:
            probeirri.append(1/(Irreaspera + totaldif))
            
        if e in dfre.index:
            probreli.append(dfre.prob[e])
        if e in dfi.index:
            probeirri.append(dfi.prob[e])
    m_irri = 1
    m_reli = 1
    for z in probeirri:
        m_irri*=z
    for j in probreli:
        m_reli*=j
    f_irre = c_irre*m_irri
    f_reli = c_rele*m_reli
    #print("Relevantes")
    #print(f_reli)
    #print("Irrelevante")
    #print(f_irre)
    if f_irre > f_reli:
        return 0
    else:
        return 1

In [144]:
print(classificador("marlboro reds"))

1


In [145]:
teste = pd.read_excel('tweetsteste.xlsx')
teste = teste.loc[:,["Teste","Relevante"]]
teste.head()

,Teste,Relevante
0,"rt @seanlynch845: final: cornwall 16, marlboro...",0
1,rt @1990sf1: ayrton senna (marlboro mclaren-ho...,1
2,game day!!!! home opener vs marlboro 230 pm.,0
3,rt @lbdfc: final score marlboro 10 leominster ...,0
4,rt @mattmcnamaratv_: cornwall with impressive ...,0


In [146]:
teste["Classificador"] = teste['Teste'].apply(classificador)
teste.head()

,Teste,Relevante,Classificador
0,"rt @seanlynch845: final: cornwall 16, marlboro...",0,0
1,rt @1990sf1: ayrton senna (marlboro mclaren-ho...,1,1
2,game day!!!! home opener vs marlboro 230 pm.,0,0
3,rt @lbdfc: final score marlboro 10 leominster ...,0,0
4,rt @mattmcnamaratv_: cornwall with impressive ...,0,0


In [161]:
def limpador(a):
    if a != 'True':
        return "False"
    else:
        return "True"
    

In [173]:
teste["Resultado"] = teste.Relevante.where(teste.Relevante != teste.Classificador, 'True')

In [166]:
teste["Resultado"] = teste["Resultado"].apply(limpador)
teste["Resultado"].value_counts()

True     170
False     30
Name: Resultado, dtype: int64

In [172]:
frequencia = (170/200)*100
frequencia

85.0

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
